In [18]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [19]:
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [5]:
df


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share,gdp_per_capita
0,Afghanistan,1850,AFG,3752993.0,0.000000e+00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.121,0.000,0.0,0.00,0.00,0.000,0.000,0.000000
1,Afghanistan,1851,AFG,3769828.0,0.000000e+00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.118,0.000,0.0,0.00,0.00,0.000,0.000,0.000000
2,Afghanistan,1852,AFG,3787706.0,0.000000e+00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.116,0.000,0.0,0.00,0.00,0.000,0.000,0.000000
3,Afghanistan,1853,AFG,3806634.0,0.000000e+00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.115,0.000,0.0,0.00,0.00,0.000,0.000,0.000000
4,Afghanistan,1854,AFG,3825655.0,0.000000e+00,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.114,0.000,0.0,0.00,0.00,0.000,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46518,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,...,0.0,0.0,0.219,0.026,0.0,115.59,28.30,0.910,9.486,1487.877848
46519,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,0.558,0.037,11.795,2.199,22.920,...,0.0,0.0,0.211,0.033,0.0,118.22,30.83,0.771,6.537,1509.106089
46520,Zimbabwe,2019,ZWE,15354606.0,0.000000e+00,0.570,0.037,11.115,-0.681,-5.772,...,0.0,0.0,0.183,0.030,0.0,117.96,30.53,0.978,8.795,0.000000
46521,Zimbabwe,2020,ZWE,15669663.0,0.000000e+00,0.570,0.036,10.608,-0.507,-4.559,...,0.0,0.0,0.194,0.030,0.0,0.00,0.00,1.006,9.481,0.000000


In [6]:
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population']!= 0, df['gdp']/ df['population'], 0)
idf = df.interactive()

In [7]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=1854, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year slider', start=1854, step=5, value=1850)

In [8]:
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['co2', 'co2_per_capita',],
    button_type='success'
)

In [9]:
continents = ['United States', 'Canada', 'Japan', 'India', 'China', 'Russia']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [10]:
co2_pipeline

In [11]:
co2_plot = co2_pipeline.hvplot(x = 'year', by='country', y=yaxis_co2,line_width=2, title="CO2 emission by contry")
co2_plot

In [12]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=1854, end=2020, step=5, value=1850)
year_slider

IntSlider(end=2020, name='Year slider', start=1854, step=5, value=1850)

In [13]:
yaxis_population = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['population'],
    button_type='success'
)

In [14]:
continents = ['United States', 'Canada', 'Japan', 'India', 'China', 'Russia']

population_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_population].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [15]:
population_plot = population_pipeline.hvplot(x = 'year', by='country', y=yaxis_population,line_width=2, title="Population of contry")
population_plot

In [16]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
co2_table

In [17]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['coal_co2', 'oil_co2', 'gas_co2'], 
    button_type='success'
)

continents_excl_world = ['United States', 'Canada', 'Japan', 'India', 'China', 'Russia']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     title='CO2 source by continent')
co2_source_bar_plot
